In [45]:
import pandas as pd

In [46]:
df = pd.read_csv('amz_sales_modified.csv')

Calculate total sales for each category

In [47]:
df['total_sales'] = df['qty'] * df['price']
total_sales_per_category = df.groupby('category')['total_sales'].sum().reset_index()
total_sales_per_category = total_sales_per_category.sort_values(by='total_sales', ascending=False)
print(total_sales_per_category)

        category  total_sales
5            Set     56105661
8          kurta     54631555
7  Western Dress     18160226
6            Top      9223047
3   Ethnic Dress      1392545
4          Saree       410581
1         Bottom       362602
0         Blouse       213323
2        Dupatta          447


Calculate Average Order Value per Month:

In [48]:
df['date'] = pd.to_datetime(df['date'])
df['order_month'] = df['date'].dt.to_period('M')
df['total_order_value'] = df['price'] * df['qty']
total_order_value_per_month = df.groupby('order_month')['total_order_value'].sum().reset_index()
avg_order_value_per_month = total_order_value_per_month.groupby('order_month')['total_order_value'].mean().reset_index()
avg_order_value_per_month['total_order_value'] = avg_order_value_per_month['total_order_value'].round(2)
avg_order_value_per_month.rename(columns={'total_order_value': 'avg_order_value'}, inplace=True)
print(avg_order_value_per_month)

  order_month  avg_order_value
0     2023-03         187410.0
1     2023-04       53581624.0
2     2023-05       45638722.0
3     2023-06       41092231.0


Top 5 Best Selling Products by Quantity:

In [49]:
total_qty_per_category = df.groupby('category')['qty'].sum().reset_index()
total_qty_per_category = total_qty_per_category.sort_values(by='qty', ascending=False)
top_5_best_selling_products = total_qty_per_category.head(5)
print(top_5_best_selling_products)

        category    qty
5            Set  45289
8          kurta  45045
7  Western Dress  13943
6            Top   9903
3   Ethnic Dress   1053


Identify Top 3 Shipping Cities by Total Revenue:

In [50]:
df['total_revenue'] = df['price'] * df['qty']
total_revenue_per_city = df.groupby('ship_city')['total_revenue'].sum().reset_index()
total_revenue_per_city = total_revenue_per_city.sort_values(by='total_revenue', ascending=False)
top_3_cities_by_revenue = total_revenue_per_city.head(3)
print(top_3_cities_by_revenue)

      ship_city  total_revenue
776   BENGALURU       12531870
2906  HYDERABAD        8929220
4795     MUMBAI        6857141


Each category's contribution to the total sales

In [51]:
df['category_sales'] = df['price'] * df['qty']
category_sales = df.groupby('category')['category_sales'].sum().reset_index()
total_sales = category_sales['category_sales'].sum()
category_sales['contribution_percentage'] = round(category_sales['category_sales'] * 100.0 / total_sales, 2)
category_sales = category_sales.sort_values(by='contribution_percentage', ascending=False)
print(category_sales)

        category  category_sales  contribution_percentage
5            Set        56105661                    39.93
8          kurta        54631555                    38.88
7  Western Dress        18160226                    12.93
6            Top         9223047                     6.56
3   Ethnic Dress         1392545                     0.99
4          Saree          410581                     0.29
1         Bottom          362602                     0.26
0         Blouse          213323                     0.15
2        Dupatta             447                     0.00


Percentage of orders made using each payment mode.

In [52]:
order_counts = df.groupby('payment_mode')['order_id'].count().reset_index()
order_counts.rename(columns={'order_id': 'order_count'}, inplace=True)
total_orders = df['order_id'].count()
order_counts['order_percentage'] = round(order_counts['order_count'] * 100.0 / total_orders, 2)
print(order_counts)

  payment_mode  order_count  order_percentage
0         card        32495             25.19
1          cod        32215             24.98
2  net banking        32025             24.83
3          upi        32240             25.00


Percentage of Orders by Month that are Over avg order value.

In [53]:
df['date'] = pd.to_datetime(df['date'])

df['order_month'] = df['date'].dt.to_period('M')
monthly_avg = df.groupby('order_month')['price'].mean().reset_index()
monthly_avg.rename(columns={'price': 'avg_order_value'}, inplace=True)


df = df.merge(monthly_avg, on='order_month')

df['above_avg'] = df['price'] > df['avg_order_value']
monthly_summary = df.groupby('order_month').agg(
    total_orders=('order_id', 'count'),
    percentage_over_avg=('above_avg', lambda x: round(100.0 * x.sum() / x.count(), 2))
).reset_index()

monthly_summary['month'] = monthly_summary['order_month'].dt.strftime('%Y-%m')
monthly_summary.drop(columns='order_month')

print(monthly_summary)

  order_month  total_orders  percentage_over_avg    month
0     2023-03           171                36.84  2023-03
1     2023-04         49067                36.11  2023-04
2     2023-05         42040                35.91  2023-05
3     2023-06         37697                52.86  2023-06


Busiest Days in Terms of Orders (Top 3):

In [54]:
df['date'] = pd.to_datetime(df['date'])
df['order_day'] = df['date'].dt.to_period('D')
order_counts = df.groupby('order_day')['order_id'].count().reset_index()
order_counts = order_counts.sort_values(by='order_id', ascending=False)
top_3_days = order_counts.head(3)
top_3_days.rename(columns={'order_day': 'order_day', 'order_id': 'order_count'})
print(top_3_days)

     order_day  order_id
33  2023-05-03      2085
32  2023-05-02      2079
34  2023-05-04      2015


Monthly Revenue Growth Rate:

In [55]:
df['date'] = pd.to_datetime(df['date'])
df['order_month'] = df['date'].dt.to_period('M')

monthly_revenue = df.groupby('order_month')['price'].sum().reset_index()
monthly_revenue.rename(columns={'price': 'total_revenue'}, inplace=True)

monthly_revenue['growth_rate_percentage'] = (
    100.0 * (monthly_revenue['total_revenue'] - monthly_revenue['total_revenue'].shift(1)) / monthly_revenue['total_revenue'].shift(1)
).round(2)

monthly_revenue['order_month'] = monthly_revenue['order_month'].dt.strftime('%Y-%m')
monthly_revenue_result = monthly_revenue[['order_month', 'total_revenue', 'growth_rate_percentage']].sort_values(by='order_month')

print(monthly_revenue_result)

  order_month  total_revenue  growth_rate_percentage
0     2023-03         206907                     NaN
1     2023-04       59506546                28660.04
2     2023-05       50506833                  -15.12
3     2023-06       45195712                  -10.52


Top 20 average Order Value by Customer Location (City):

In [56]:
df['date'] = pd.to_datetime(df['date'])

df['order_month'] = df['date'].dt.to_period('M')
df['total_order_value'] = df['price'] * df['qty']
avg_price_citywise = df.groupby(['ship_city', 'order_month']).agg(total_order_value=('total_order_value', 'sum')).reset_index()

avg_price_citywise['avg_order_value'] = avg_price_citywise.groupby('ship_city')['total_order_value'].transform('mean').round(2)

top_20_avg_order_cities = avg_price_citywise.sort_values(by='avg_order_value', ascending=False).head(20)
print(top_20_avg_order_cities)

      ship_city order_month  total_order_value  avg_order_value
1259  BENGALURU     2023-04            4584123       3132967.50
1258  BENGALURU     2023-03              16586       3132967.50
1261  BENGALURU     2023-06            3975493       3132967.50
1260  BENGALURU     2023-05            3955668       3132967.50
4609  HYDERABAD     2023-03               7524       2232305.00
4611  HYDERABAD     2023-05            3028863       2232305.00
4612  HYDERABAD     2023-06            2887028       2232305.00
4610  HYDERABAD     2023-04            3005805       2232305.00
7611     MUMBAI     2023-06            1953803       1714285.25
7609     MUMBAI     2023-04            2616413       1714285.25
7608     MUMBAI     2023-03               5995       1714285.25
7610     MUMBAI     2023-05            2280930       1714285.25
8505  NEW DELHI     2023-03              10741       1571867.25
8506  NEW DELHI     2023-04            2520961       1571867.25
8507  NEW DELHI     2023-05            1